In [1]:
import numpy as np 
import pandas as pd 
import cv2
import os, shutil
from PIL import Image
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [2]:
Targets = ['abdomen', 'arm', 'head', 'legs']
Target_select = 2  # 0 to 3 selections

conseder_side = 1

datalink = {
    'train': '/kaggle/input/ultrasound-fpu23-csv-files/train_labels.csv',
    'val': '/kaggle/input/ultrasound-fpu23-csv-files/val_labels.csv',
    'test' : '/kaggle/input/ultrasound-fpu23-csv-files/test_labels.csv'
}

root_dir = '/kaggle/input/source-fpus23/Dataset/four_poses'

def create_dir(folderlocation):
    if not os.path.exists(folderlocation):
        os.mkdir(folderlocation)
destination_folder = './zzside/'
create_dir(destination_folder)

for phase in ['train/','test/','val/']:
    create_dir(destination_folder+phase)
    create_dir(destination_folder+phase+Targets[Target_select])
    create_dir(destination_folder+phase+'other')
    if phase == 'test/':
        continue
    if conseder_side:
        create_dir(destination_folder+phase+'zzside')

In [3]:
for phase in ['train','test','val']:
    csv_frame = pd.read_csv(datalink[phase])
    print(len(csv_frame))
    for idx in range(len(csv_frame)):
        sub_dirs = csv_frame.iloc[idx, 0]
        image_source = root_dir + sub_dirs +'/' + csv_frame.iloc[idx, 1]
        
        labels = csv_frame.iloc[idx, 3]
    
        image_destination = destination_folder+phase+'/other/' + sub_dirs + csv_frame.iloc[idx, 1]
    
        if Targets[Target_select] in labels:
            image_destination = destination_folder+ phase+ '/'+Targets[Target_select] + '/'+sub_dirs + csv_frame.iloc[idx, 1]
        
        shutil.copy(image_source, image_destination)

9067
3021
3022


In [4]:
annotation_box_dir = '/kaggle/input/source-fpus23/Dataset/boxes/annotation/'
image_source_dir = '/kaggle/input/source-fpus23/Dataset/four_poses/'


print('Organizing Data for '+Targets[Target_select] +' Detection')


def patchmaker(xbr,xtl,ybr,ytl, edgecolor_ = 'r'):
    return Rectangle((xtl,ytl),xbr-xtl,ybr-ytl,linewidth=1,edgecolor=edgecolor_,facecolor='none')

Organizing Data for head Detection


In [5]:
if conseder_side:
    subdirs = [x[0] for x in os.walk(annotation_box_dir)]
    #print(subdirs)
    
    split_variable = 0
    
    for sub_dir in subdirs[1:]:
        plane = sub_dir.split("/")[-1]
        
        with open(sub_dir + '/annotations.xml', 'r') as f:
            data = f.read()
    
            Bs_data = BeautifulSoup(data, "xml")
            b_image = Bs_data.find_all('image')
            print('Folder Name:',plane,'Number of image annotation:',len(b_image))
            for image_index in range(len(b_image)):
                boxes = b_image[image_index].find_all('box')
                image_source = image_source_dir + plane + '/' + b_image[image_index].get('name')
              
                   
                img = mpimg.imread(image_source)
                if len(boxes) ==1:   
                    xbr = float(boxes[0].get('xbr'))
                    ybr = float(boxes[0].get('ybr'))
                    xtl = float(boxes[0].get('xtl'))
                    ytl = float(boxes[0].get('ytl'))
                    rect = patchmaker(xbr,xtl,ybr,ytl)
                    
                    img1 = Image.fromarray( (img[int(ytl):int(ybr), int(2*xtl/3):int(xtl),:]* 255).astype(np.uint8) )
                    img2 = Image.fromarray( (img[int(ytl):int(ybr), int(xbr):int((img.shape[1]+xbr)/2),:]* 255).astype(np.uint8) )
    
                    split_variable = split_variable +1
                    if split_variable == 3:
                        img1.save(destination_folder+'val/zzside/i1'+str(split_variable)+b_image[image_index].get('name'), format="PNG")
                        #img2.save(destination_folder+'test/zzside/i2'+str(split_variable)+b_image[image_index].get('name'), format="PNG")
                        
                    img1.save(destination_folder+'train/zzside/i1'+str(split_variable)+b_image[image_index].get('name'), format="PNG")
                    img2.save(destination_folder+'train/zzside/i2'+str(split_variable)+b_image[image_index].get('name'), format="PNG")
                    if split_variable >1:
                        break
    
    image_iter = np.zeros([50, 50, 3])
    
    for iter1 in range(6):
        for iter2 in range(6):
            for iter3 in range(6):
                image_iter[:,:,0] = iter1*50
                image_iter[:,:,1] = iter2*50
                image_iter[:,:,2] = iter3*50
                
                cv2.imwrite(destination_folder+'train/zzside/' + "Monochrome_"+str(iter1)+str(iter2)+str(iter3)+".png", image_iter)

Folder Name: stream_huvb_aroundhead_v Number of image annotation: 305
Folder Name: stream_huvb_longrec_h Number of image annotation: 597
Folder Name: stream_huvf_aroundabd_h Number of image annotation: 394
Folder Name: stream_hdvf_longrec_v Number of image annotation: 339
Folder Name: stream_hdvb_longrec_h Number of image annotation: 378
Folder Name: stream_huvf_longrec_h Number of image annotation: 1083
Folder Name: stream_hdvb_aroundarms_h Number of image annotation: 312
Folder Name: stream_hdvb_aroundabd_h Number of image annotation: 361
Folder Name: stream_huvb_aroundarms_h Number of image annotation: 224
Folder Name: stream_hdvf_aroundlegs_h Number of image annotation: 285
Folder Name: stream_huvb_longrec_v Number of image annotation: 892
Folder Name: stream_hdvf_longrec_h Number of image annotation: 556
Folder Name: stream_hdvf_aroundarms_v Number of image annotation: 206
Folder Name: stream_huvf_aroundarms_v Number of image annotation: 308
Folder Name: stream_huvf_aroundlegs_h N

[ WARN:0@249.626] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


In [6]:
shutil.make_archive('Side_data', 'zip', destination_folder)
shutil.rmtree(destination_folder)